In [ ]:
using DynamicPolynomials
using SumOfSquares
using CSDP
using JuMP
using Plots
using LinearAlgebra
using PyCall

# 0. Init system variable and parameters

In [ ]:
#init state and input variables
@polyvar x[1:2]
@polyvar u[1:2]

#define system dynamics
vectorField = [ 0.5*x[1] - 0.2*x[2]  - 0.01 * x[1] * x[2] - 0.5 *u[1] + 0.5 * u[2], 
               x[1] -0.4*x[2] - 0.05 * x[2]^2  - 0.7 *u[2] + 0.1 * u[1] ] 

#define unsafe set (obstacle)
g = 0.1^2 - x[1]^2 - x[2]^2

#state and input bounds
bounds = [[-10., 10.], [-10., 10.] ]

In [ ]:
function prepare_domain(x::Vector{<:Variable}, bounds::Vector{Vector{Float64}})
   poly_list = [ 
        @set(x[i] - l >= 0) ∩ @set(u - x[i] >= 0 ) ∩ @set((x[i] -l)*(u-x[i]) >= 0)
        for (i, (l, u)) in enumerate(bounds)
        ] 
   poly_list
end

In [ ]:
#instantiate parameters
λ = 1
ϵ = 1
max_degree=4
U = [[-0.05, -0.05], [-0.05, 0.05], [0.05, -0.05], [0.05, 0.05]]

In [ ]:
# generate test points
function get_random(limits::Vector{Vector{Float64}}, g::Polynomial)
    function get_random_scalar(lb, ub )
        lb + rand()*(ub - lb) 
    end
    while (true)
        pt = [get_random_scalar(l[1], l[2]) for l in limits]
        if g(pt...) >= 0
            continue
        else
            return pt
        end
    end
end

test_pts = [ get_random(bounds, g) for _ in 1:500];

# 1. Computing initial set of barriers for each input U_i

In [ ]:
# function to generate initial barriers for each input

function generate_barrier(x, u, bounds, g, vectorField, U, test_pts; max_degree=4,ϵ = 0.5, λ = 0.1, γ = 10.)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
#     println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    B = polynomial(c[1:end], monos) 
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=dom)
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    #stat = JuMP.primal_status(model)
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), lm
end

In [ ]:
elapsed1 = @elapsed begin
B_00, lm00 = generate_barrier(x, u, bounds, g, vectorField, U[1], test_pts)
end
display(B_00)
test_pts = filter!(pt -> B_00(pt...) <= 0., test_pts)

In [ ]:
elapsed2 = @elapsed begin
B_01, lm01 = generate_barrier(x, u, bounds, g, vectorField, U[2], test_pts)
end
display(B_01)
test_pts = filter!(pt -> B_01(pt...) <= 0., test_pts)

In [ ]:
elapsed3 = @elapsed begin
B_10, lm10 = generate_barrier(x, u, bounds, g, vectorField, U[3], test_pts)
end
display(B_10)
test_pts = filter!(pt -> B_10(pt...) <= 0., test_pts)

In [ ]:
elapsed4 = @elapsed begin
B_11, lm11 = generate_barrier(x, u, bounds, g, vectorField, U[4], test_pts)
end
display(B_11)
test_pts = filter!(pt -> B_11(pt...) <= 0., test_pts)

# 2. Computing Successive Barriers

In [ ]:
# functions to compute successive barriers

In [ ]:
function generate_barrier(x, u, bounds, g, vectorField, U, test_pts; max_degree=4,ϵ = 0.25, λ = 0.1, γ = 10.)
    solver = optimizer_with_attributes(CSDP.Optimizer)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
#     println("Domain: $dom")
    # negative inside the obstacle
    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    B = polynomial(c[1:end], monos) 
    #negative inside the domain
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=dom)
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), lm
end

function generate_successive_barrier(x::Vector{<:Variable}, u::Vector{<:Variable}, 
        bounds::Vector{Vector{Float64}}, g::Polynomial, 
        vectorField::Vector{<:Polynomial}, U::Vector{Float64}, 
        test_pts::Vector{Vector{Float64}}, ancestors::Vector{<:Polynomial}, 
        pt_to_eliminate::Vector{Float64}; 
        max_degree=4,ϵ = 1, λ = 1, γ = 10.)
    solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
    model = SOSModel(solver)
    dom_list = prepare_domain(x, bounds)
    dom = reduce( (s1, s2) -> s1 ∩ s2, dom_list)
#     println("Domain: $dom")

    monos = monomials(x, 0:max_degree)
    N = length(monos) 
    @variable(model, -γ <= c[1:N] <= γ)
    B = polynomial(c[1:end], monos) 
    # negative inside the obstacle
    @constraint(model, cons1, B <= -ϵ, domain=dom ∩ @set(g >= 0) )
    
    # dynamics constraints
    B_dot = dot(differentiate(B,x), vectorField)
    B_dot_with_u = subs(B_dot, u => U)
    if size(ancestors)[1] >= 1
        new_domain = dom ∩ (reduce(∩, [@set(B <= 0.) for B in ancestors]))
    else
        new_domain = dom 
    end
    @constraint(model, cons2, B_dot_with_u >= λ * B, domain=new_domain)
    
    # eliminate the point we would like to eliminate
    @constraint(model, B(pt_to_eliminate...) >= ϵ)
    set_objective_sense(model, MOI.FEASIBILITY_SENSE)
    
    # maximize the sum of values for all test points
    objective_fn = sum([B(pt...) for pt in test_pts])
    @objective(model, Max, objective_fn) # keep as many points outside the barrier as you can
    JuMP.optimize!(model)
    stat = JuMP.primal_status(model)
    if stat != FEASIBLE_POINT
        return missing
    end
    # found feasible point
    println(solution_summary(model))
    lm = [lagrangian_multipliers(cons1)]
    push!(lm, lagrangian_multipliers(cons2))
    value(B), lm
end

function compute_next_level_barriers(x::Vector{<:Variable}, u::Vector{<:Variable}, bounds::Vector{Vector{Float64}}, 
                                    g::Polynomial, vectorField::Vector{<:Polynomial}, 
                                    u_val::Vector{Vector{Float64}}, test_pts::Vector{Vector{Float64}}, 
                                    ancestors::Vector{<:Polynomial}) 
    eliminated = []
    second_level_barriers=[]
    for (j,pt) in enumerate(test_pts) 
        if j in eliminated
            # ignore the ones already eliminated
            continue
        end
        # go through each control 
        for u_val in U
            # generate a barrier 
            B = generate_successive_barrier(x, u, bounds, g, vectorField, u_val, test_pts, ancestors, pt)
            # check if we found something
            if !ismissing(B)
                println("Bingo: Found $B")
                useful = false # check if it is actually useful
                for (k,pt_new) in enumerate(test_pts)
                    if k in eliminated
                        continue
                    end
                   if (B[1](pt_new...) >= 0.)
                    push!(eliminated, k)   
                    println("\t eliminated $pt_new")
                    useful=true
                   end
                end
                if (useful)
                    println("Num remaining = $(size(test_pts)[1] - size(eliminated)[1])")
                    push!(second_level_barriers, (B[1], u_val, B[2]))
                    break
                end
            end
        end
    end
    new_test_pts = [pt for (j, pt) in enumerate(test_pts) if !(j in eliminated)]
    return (second_level_barriers, new_test_pts)
end
test_pts = [ get_random(bounds, g) for _ in 1:25];

In [ ]:
zero_level_barriers = [(B_00,U[1]),(B_01,U[2]),(B_10,U[3]),(B_11,U[4])]
print(size(zero_level_barriers))
ancestors = [B_00, B_01, B_10, B_11]
s_elapsed = @elapsed begin
(first_level_barriers, test_pts_1) = compute_next_level_barriers(x, u, bounds, g, vectorField, U, test_pts, ancestors)
end

In [ ]:
ancestors1 = [B_00, B_01, B_10, B_11]
ancestors2 = [B for (B,_) in first_level_barriers]
ancestors = vcat(ancestors1, ancestors2)
test_pts_1 = [ get_random(bounds, g) for _ in 1:25];
(second_level_barriers, test_pts_2) = compute_next_level_barriers(x, u, bounds, g, vectorField, U, test_pts_1, ancestors)

# 3. Plots

In [ ]:
# improvement in the control invariant region at level 3

using Plots

function make_contour_plot(B, limits,clr, legend="")
    x = range(limits[1], limits[2], length=500)
    y = range(limits[1], limits[2], length=500)
    z = @. B(x', y)
    contour!(x, y, z,levels=[0.0, 0.0], color=clr, colorbar=false, labels=[legend])
end

limits=(-3., 3.)
plot(xlims=limits, ylims=limits,showaxis = false)
rectangle(w, h, x, y) = Shape(x .+ [0,w,w,0], y .+ [0,0,h,h])
for x in -10.:.1:10
    for y in -10:0.1:10
        if ( B_00(x,y) > 0. || B_01(x,y) > 0.)
            plot!(rectangle(0.1, 0.1, x, y), label=false, fill=:seagreen1, linecolor=:seagreen1,showaxis = false)
        else
            if (any( B(x,y) > 0. for (B,_) in first_level_barriers ) )
                plot!(rectangle(0.1, 0.1, x, y), label=false, fill=:limegreen, linecolor=:limegreen,showaxis = false)
            else
                if (any(B(x,y) > 0. for (B,_) in second_level_barriers))
                   plot!(rectangle(0.1, 0.1, x, y), label=false, fill=:darkgreen, linecolor=:darkgreen,showaxis = false)
                end
            end
        end
    end
end
make_contour_plot(g, limits,"black")
plot!(xlims=limits, ylims=limits,showaxis = false)
savefig("figures/Figure6a-example-2d-successive-approx.png")
plot!(xlims=limits, ylims=limits,showaxis = false)


# 4. Benchmarks

In [ ]:
# generating data for Table 1 in the paper (row for current system)

In [ ]:
println("Time taken for B1: $(elapsed1+elapsed2+elapsed3+elapsed4)")
println("# barriers B1: $(size(zero_level_barriers))")
println("Time taken for B2: $(s_elapsed)")
println("# barriers B2: $(size(first_level_barriers))")

In [ ]:
# generating test points with fixed seed for evaluations (Table 2 in paper)

function generate_random_points_fixed_seed(limits::Vector{Vector{Float64}}, g::Polynomial)
     function get_random_scalar(lb, ub )
        Random.seed!(1234)
        lb + rand()*(ub - lb) 
     end
     while (true)
         pt = [get_random_scalar(l[1], l[2]) for l in limits]
         if g(pt[1], pt[2]) >= 0
             continue
         else
             return pt
         end
     end
end
test_pts_fixed_seed = [ get_random(bounds, g) for _ in 1:1000];

In [ ]:
# functions for FOSSIL comparison (Table 2 in paper)

py"""
import math
def eval_fossil_barrier(x0,x1):
    barrier = (0.2642662525177002 + 0.32199999690055847 * pow((-0.39800000190734863 + (-0.46599999070167542 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (-0.37900000810623169 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (-0.375 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (-0.29199999570846558 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (-0.29199999570846558 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (-0.2800000011920929 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (-0.25699999928474426 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (-0.1940000057220459 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (-0.19200000166893005 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (-0.096000000834465027 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1))))), 2) - 0.014000000432133675 * pow((-0.20800000429153442 + (-0.44299998879432678 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (-0.41200000047683716 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (-0.37299999594688416 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (-0.125 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (-0.11900000274181366 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (-0.048999998718500137 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.55299997329711914 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (0.625 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (0.76599997282028198 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (0.87000000476837158 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1))))), 2) + 0.22300000488758087 * pow((-0.18299999833106995 + (-0.40200001001358032 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (-0.3970000147819519 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (-0.3619999885559082 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (-0.30799999833106995 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (-0.24799999594688416 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (-0.22800000011920929 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (-0.1940000057220459 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (-0.17100000381469727 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (-0.15000000596046448 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (-0.094999998807907104 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1))))), 2) - 0.041999999433755875 * pow((-0.035999998450279236 + (-0.23100000619888306 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (-0.16200000047683716 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (-0.10700000077486038 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (-0.081000000238418579 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (-0.014000000432133675 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.34799998998641968 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.36000001430511475 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (0.38299998641014099 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (0.50499999523162842 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (0.62800002098083496 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1))))), 2) + 0.23399999737739563 * pow((-0.023000000044703484 + (-0.50499999523162842 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (-0.21400000154972076 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (-0.12700000405311584 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (-0.045000001788139343 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (0.057999998331069946 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (0.072999998927116394 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (0.081000000238418579 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.093000002205371857 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (0.12800000607967377 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.335999995470047 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1))))), 2) - 0.59700000286102295 * pow((0.0010000000474974513 + (-0.6940000057220459 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (-0.5820000171661377 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (-0.54400002956390381 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (-0.45100000500679016 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (-0.28499999642372131 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.21500000357627869 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.55199998617172241 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (0.71100002527236938 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (0.9869999885559082 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (1.156999945640564 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1))))), 2) + 0.18999999761581421 * pow((0.029999999329447746 + (0.012000000104308128 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (0.082000002264976501 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (0.18799999356269836 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (0.2070000022649765 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.27700001001358032 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (0.31999999284744263 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (0.43000000715255737 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (0.44999998807907104 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (0.53799998760223389 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.69999998807907104 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1))))), 2) + 0.27399998903274536 * pow((0.33000001311302185 + (0.02199999988079071 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (0.035999998450279236 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.090999998152256012 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (0.1550000011920929 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (0.17299999296665192 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (0.22699999809265137 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (0.29699999094009399 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (0.33100000023841858 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (0.40200001001358032 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.5 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1))))), 2) + 0.24099999666213989 * pow((0.42599999904632568 + (0.10700000077486038 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.1080000028014183 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (0.12099999934434891 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (0.14300000667572021 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (0.23199999332427979 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (0.37200000882148743 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (0.40400001406669617 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.42800000309944153 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (0.48600000143051147 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1)))) + (0.56099998950958252 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1))))), 2) + 0.23199999332427979 * pow((0.45100000500679016 + (-0.14800000190734863 / (1 + math.exp((1.2220000028610229 + 0.53299999237060547 * x0 - 0.38400000333786011 * x1)))) + (-0.0030000000260770321 / (1 + math.exp((1.1599999666213989 + 0.60699999332427979 * x0 + 0.30199998617172241 * x1)))) + (0.017999999225139618 / (1 + math.exp((0.72399997711181641 - 0.60100001096725464 * x0 - 1.3020000457763672 * x1)))) + (0.064999997615814209 / (1 + math.exp((0.35100001096725464 + 0.85699999332427979 * x0 - 0.017999999225139618 * x1)))) + (0.089000001549720764 / (1 + math.exp((0.29100000858306885 - 1.2630000114440918 * x0 - 0.4699999988079071 * x1)))) + (0.1289999932050705 / (1 + math.exp((0.63599997758865356 - 1.2940000295639038 * x0 - 0.45199999213218689 * x1)))) + (0.13500000536441803 / (1 + math.exp((0.27399998903274536 + 1.0039999485015869 * x0 + 0.050999999046325684 * x1)))) + (0.23800000548362732 / (1 + math.exp((0.39100000262260437 + 1.0089999437332153 * x0 + 0.1550000011920929 * x1)))) + (0.53600001335144043 / (1 + math.exp((0.21400000154972076 - 0.80900001525878906 * x0 - 0.27700001001358032 * x1)))) + (0.64099997282028198 / (1 + math.exp((0.36500000953674316 - 1.2089999914169312 * x0 - 0.53700000047683716 * x1))))), 2))
    return barrier
"""
fossil_eval = py"eval_fossil_barrier";

function B_lex(state, zero_level_barriers, one_level_barriers, two_level_barriers)
   b0, u0 = maximum([ (B(state...), u) for (B, u) in zero_level_barriers])
   if (b0 >= 0)
        return b0, u0
    end
   b1, u1 = maximum([ (B(state...), u) for (B, u) in one_level_barriers])
   if (b1 >= 0)
        return b1, u1
    end
   b2, u2 = maximum([ (B(state...), u) for (B, u) in two_level_barriers])
   if (b2 >= 0)
        return b2, u2
   end
   return max((b0, u0), (b1, u1), (b2, u2))
end

function fossil_comparisons(zero_level_barriers,first_level_barriers,second_level_barriers, fossil_eval, test_pts_fixed_seed)
    counter = 0
    counter_f = 0
    counter_s = 0
    counter_x = 0
    for pt in test_pts_fixed_seed
        c,_ = B_lex(pt, zero_level_barriers, first_level_barriers, second_level_barriers )
        if (fossil_eval(pt[1],pt[2]))>=0 && -c < 0
            counter = counter + 1
        end
        if (fossil_eval(pt[1],pt[2]))<0
            counter_f = counter_f + 1
        end
        if (-c)<0
            counter_s = counter_s + 1
        end
        if (fossil_eval(pt[1],pt[2]))<0 && -c >= 0
            counter_x = counter_x + 1
        end
    end
    println("Total test points: 1000")
    println("Safe in FOSSIL: $(counter)")
    println("Safe in Ours: $(counter_s)")
    println("Unsafe in FOSSIL and Safe in Ours: $(counter_f)")
    println("Safe in FOSSIL and Unsafe in Ours: $(counter_x)")
    
    return counter_f, counter_s, counter, counter_x
end

In [ ]:
fossil_comparisons(zero_level_barriers,first_level_barriers,second_level_barriers, fossil_eval, test_pts_fixed_seed)

# 5. Verification

In [ ]:
"""
Certifying barriers using constraint PSD check
"""

using DelimitedFiles

function check_psd_constraints(multipliers_array::Vector)
    counter = 0
    M = []
    for mults in multipliers_array
        for lms in mults
            for lm in lms
                if !isposdef(Matrix(lm.Q)) && !isposdef(Diagonal(svd(Matrix(lm.Q)).S))
                    counter += 1
                else 
#                     display(Matrix(lm.Q))
                    push!(M, [Matrix(lm.Q)])
                end
            end
        end
    end
    open("matrices_p3.txt", "w") do io
        writedlm(io, M)
    end
    if counter==0
        println("All constraints are PSD: Barrier certified")
    else
        println("PSD check failed!")
    end
end

In [ ]:
all_barrier_plus = push!([first_level_barriers], second_level_barriers);

In [ ]:
lms=[]
for i in all_barrier_plus
    if (!ismissing(i[1][3]))
        lms = append!(lms,i[1][3])
    end
end

In [ ]:
check_psd_constraints([lm00,lm01,lm10,lm11,lms])